<a href="https://colab.research.google.com/github/santhimaddipudi/Python-work/blob/master/Polars_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##String Manipulation

In [ ]:
url = 'https://github.com/mattharrison/datasets/raw/' \
  'master/data/__mharrison__2020-2021.csv'
import polars as pl
raw=pl.read_csv(url)
print(raw)


In [ ]:
#select only few columns
def tweak_twit(df):
          return (df .select(['Tweet id', 'Tweet permalink', 'Tweet text', 'time',
                              'impressions', 'engagements', 'engagement rate',
                              'retweets', 'replies', 'likes', 'user profile clicks'])
                 )
twit = tweak_twit(raw)
print(twit)

In [7]:
#.str accessor, much like Pandas. This accessor provides access to string manipulation methods
col = pl.col('Tweet permalink')
print([m for m in dir(col.str) if not m.startswith('_')])

['concat', 'contains', 'contains_any', 'count_matches', 'decode', 'encode', 'ends_with', 'escape_regex', 'explode', 'extract', 'extract_all', 'extract_groups', 'extract_many', 'find', 'find_many', 'head', 'join', 'json_decode', 'json_path_match', 'len_bytes', 'len_chars', 'normalize', 'pad_end', 'pad_start', 'replace', 'replace_all', 'replace_many', 'reverse', 'slice', 'split', 'split_exact', 'splitn', 'starts_with', 'strip_chars', 'strip_chars_end', 'strip_chars_start', 'strip_prefix', 'strip_suffix', 'strptime', 'tail', 'to_date', 'to_datetime', 'to_decimal', 'to_integer', 'to_lowercase', 'to_time', 'to_titlecase', 'to_uppercase', 'zfill']


In [8]:
#links don’t start with the correct prefix in the column
col = pl.col('Tweet permalink')
print(twit.filter(~col.str.starts_with('https://twitter.com/')))

shape: (0, 11)
┌──────────┬───────────┬────────────┬──────┬───┬──────────┬─────────┬───────┬──────────────────────┐
│ Tweet id ┆ Tweet     ┆ Tweet text ┆ time ┆ … ┆ retweets ┆ replies ┆ likes ┆ user profile clicks  │
│ ---      ┆ permalink ┆ ---        ┆ ---  ┆   ┆ ---      ┆ ---     ┆ ---   ┆ ---                  │
│ i64      ┆ ---       ┆ str        ┆ str  ┆   ┆ f64      ┆ f64     ┆ f64   ┆ f64                  │
│          ┆ str       ┆            ┆      ┆   ┆          ┆         ┆       ┆                      │
╞══════════╪═══════════╪════════════╪══════╪═══╪══════════╪═════════╪═══════╪══════════════════════╡
└──────────┴───────────┴────────────┴──────┴───┴──────────┴─────────┴───────┴──────────────────────┘


In [9]:
print('https://metasnake.com/effective-polars'.split('/'))

['https:', '', 'metasnake.com', 'effective-polars']


In [ ]:
print(twit.select(pl.col('Tweet permalink').str.split('/').list.get(3)))

In [20]:
print(twit.select(col.str.split('/').list.len()))

shape: (5_791, 1)
┌─────────────────┐
│ Tweet permalink │
│ ---             │
│ u32             │
╞═════════════════╡
│ 6               │
│ 6               │
│ 6               │
│ 6               │
│ 6               │
│ …               │
│ 6               │
│ 6               │
│ 6               │
│ 6               │
│ 6               │
└─────────────────┘


In [10]:
#.list.to_struct method converts each list into a struct.
#unnest method to convert the struct into columns
print(twit
 .select(col.str.split('/')
 .list.to_struct())
 .unnest('Tweet permalink')
 )

shape: (5_791, 6)
┌─────────┬─────────┬─────────────┬───────────────┬─────────┬─────────────────────┐
│ field_0 ┆ field_1 ┆ field_2     ┆ field_3       ┆ field_4 ┆ field_5             │
│ ---     ┆ ---     ┆ ---         ┆ ---           ┆ ---     ┆ ---                 │
│ str     ┆ str     ┆ str         ┆ str           ┆ str     ┆ str                 │
╞═════════╪═════════╪═════════════╪═══════════════╪═════════╪═════════════════════╡
│ https:  ┆         ┆ twitter.com ┆ __mharrison__ ┆ status  ┆ 1212580517905780737 │
│ https:  ┆         ┆ twitter.com ┆ __mharrison__ ┆ status  ┆ 1212582494828036097 │
│ https:  ┆         ┆ twitter.com ┆ __mharrison__ ┆ status  ┆ 1212613735698690049 │
│ https:  ┆         ┆ twitter.com ┆ __mharrison__ ┆ status  ┆ 1212911749617242113 │
│ https:  ┆         ┆ twitter.com ┆ __mharrison__ ┆ status  ┆ 1212920556028252160 │
│ …       ┆ …       ┆ …           ┆ …             ┆ …       ┆ …                   │
│ https:  ┆         ┆ twitter.com ┆ __mharrison__ ┆ status

<ipython-input-10-40fa60258701>:3: UserWarning: `to_struct()` should be passed a list of field names to avoid query errors in subsequent operations (e.g. <struct operation> not supported for dtype Unknown)
  .list.to_struct())


In [11]:
#Extract username with regular expression
regex = r'^https:\/\/twitter\.com\/([a-zA-Z0-9_]+)\/status\/(\d+)$'
print(twit.select(user=col.str.extract(regex, group_index=1))
)

shape: (5_791, 1)
┌───────────────┐
│ user          │
│ ---           │
│ str           │
╞═══════════════╡
│ __mharrison__ │
│ __mharrison__ │
│ __mharrison__ │
│ __mharrison__ │
│ __mharrison__ │
│ …             │
│ __mharrison__ │
│ __mharrison__ │
│ __mharrison__ │
│ __mharrison__ │
│ __mharrison__ │
└───────────────┘


In [ ]:
#Get word count from tweet text
tweet_col = pl.col('Tweet text')
print(twit
 .with_columns(word_count=tweet_col.str.split(' ').list.len()))

In [17]:
print(twit
 .with_columns(upper=tweet_col.str.to_titlecase()))#Convert strings to title case

shape: (5_791, 12)
┌────────────┬────────────┬────────────┬────────────┬───┬─────────┬───────┬────────────┬───────────┐
│ Tweet id   ┆ Tweet      ┆ Tweet text ┆ time       ┆ … ┆ replies ┆ likes ┆ user       ┆ upper     │
│ ---        ┆ permalink  ┆ ---        ┆ ---        ┆   ┆ ---     ┆ ---   ┆ profile    ┆ ---       │
│ i64        ┆ ---        ┆ str        ┆ str        ┆   ┆ f64     ┆ f64   ┆ clicks     ┆ str       │
│            ┆ str        ┆            ┆            ┆   ┆         ┆       ┆ ---        ┆           │
│            ┆            ┆            ┆            ┆   ┆         ┆       ┆ f64        ┆           │
╞════════════╪════════════╪════════════╪════════════╪═══╪═════════╪═══════╪════════════╪═══════════╡
│ 1212580517 ┆ https://tw ┆ Sounds     ┆ 2020-01-02 ┆ … ┆ 0.0     ┆ 3.0   ┆ 3.0        ┆ Sounds    │
│ 905780737  ┆ itter.com/ ┆ like a     ┆ 03:44:00+0 ┆   ┆         ┆       ┆            ┆ Like A    │
│            ┆ __mharriso ┆ great      ┆ 0:00       ┆   ┆         ┆     

##Aggregation with Polars

In [19]:
import pandas as pd
df=pd.read_csv("/content/sample_data/vehicles.csv")
df_polars = pl.from_pandas(df)

<ipython-input-19-8d0bdcca8851>:2: DtypeWarning: Columns (74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/content/sample_data/vehicles.csv")


In [20]:
col = pl.col('make')
print(sorted(att for att in dir(col) if not att.startswith('_')))

['abs', 'add', 'agg_groups', 'alias', 'all', 'and_', 'any', 'append', 'approx_n_unique', 'arccos', 'arccosh', 'arcsin', 'arcsinh', 'arctan', 'arctanh', 'arg_max', 'arg_min', 'arg_sort', 'arg_true', 'arg_unique', 'arr', 'backward_fill', 'bin', 'bitwise_and', 'bitwise_count_ones', 'bitwise_count_zeros', 'bitwise_leading_ones', 'bitwise_leading_zeros', 'bitwise_or', 'bitwise_trailing_ones', 'bitwise_trailing_zeros', 'bitwise_xor', 'bottom_k', 'bottom_k_by', 'cast', 'cat', 'cbrt', 'ceil', 'clip', 'cos', 'cosh', 'cot', 'count', 'cum_count', 'cum_max', 'cum_min', 'cum_prod', 'cum_sum', 'cumulative_eval', 'cut', 'degrees', 'deserialize', 'diff', 'dot', 'drop_nans', 'drop_nulls', 'dt', 'entropy', 'eq', 'eq_missing', 'ewm_mean', 'ewm_mean_by', 'ewm_std', 'ewm_var', 'exclude', 'exp', 'explode', 'extend_constant', 'fill_nan', 'fill_null', 'filter', 'first', 'flatten', 'floor', 'floordiv', 'forward_fill', 'from_json', 'gather', 'gather_every', 'ge', 'get', 'gt', 'has_nulls', 'hash', 'head', 'hist'

In [24]:
df_polars

barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,…,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,baseModel,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,i64,i64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,str,i64,str,i64,i64,i64,str,str,i64,i64,i64,f64,i64,…,str,str,bool,i64,i64,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,i64,i64,i64
14.167143,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,"""Rear-Wheel Drive""",9011,"""(FFS)""",-1,2300,0,"""Regular""","""Regular Gasoline""",-1,-1,25,0.0,0,…,"""Spider Veloce 2000""","""Y""",false,0,0,0,0.0,0.0,0.0,0.0,"""Manual 5-spd""",23.3333,0.0,35.0,0.0,"""Two Seaters""",1985,-2750,"""Spider""",null,null,null,null,null,null,null,null,null,null,0.0,null,"""Tue Jan 01 00:00:00 EST 2013""","""Tue Jan 01 00:00:00 EST 2013""",null,0,0,0
27.046364,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,"""Rear-Wheel Drive""",22020,"""(GUZZLER)""",-1,4400,0,"""Regular""","""Regular Gasoline""",-1,-1,14,0.0,0,…,"""Testarossa""","""N""",false,0,0,0,0.0,0.0,0.0,0.0,"""Manual 5-spd""",11.0,0.0,19.0,0.0,"""Two Seaters""",1985,-13250,"""Testarossa""","""T""",null,null,null,null,null,null,null,null,null,0.0,null,"""Tue Jan 01 00:00:00 EST 2013""","""Tue Jan 01 00:00:00 EST 2013""",null,0,0,0
11.018889,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,"""Front-Wheel Drive""",2100,"""(FFS)""",-1,1800,0,"""Regular""","""Regular Gasoline""",-1,-1,33,0.0,0,…,"""Charger""","""Y""",false,0,0,0,0.0,0.0,0.0,0.0,"""Manual 5-spd""",29.0,0.0,47.0,0.0,"""Subcompact Cars""",1985,-250,"""Charger""",null,"""SIL""",null,null,null,null,null,null,null,null,0.0,null,"""Tue Jan 01 00:00:00 EST 2013""","""Tue Jan 01 00:00:00 EST 2013""",null,0,0,0
27.046364,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,"""Rear-Wheel Drive""",2850,null,-1,4400,0,"""Regular""","""Regular Gasoline""",-1,-1,12,0.0,0,…,"""B150/B250 Wagon 2WD""","""N""",false,0,0,0,0.0,0.0,0.0,0.0,"""Automatic 3-spd""",12.2222,0.0,16.6667,0.0,"""Vans""",1985,-13250,"""B150/B250 Wagon""",null,null,null,null,null,null,null,null,null,null,0.0,null,"""Tue Jan 01 00:00:00 EST 2013""","""Tue Jan 01 00:00:00 EST 2013""",null,0,0,0
15.658421,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,"""4-Wheel or All-Wheel Drive""",66031,"""(FFS,TRBO)""",-1,3300,0,"""Premium""","""Premium Gasoline""",-1,-1,23,0.0,0,…,"""Legacy AWD Turbo""","""N""",false,0,90,0,0.0,0.0,0.0,0.0,"""Manual 5-spd""",21.0,0.0,32.0,0.0,"""Compact Cars""",1993,-7750,"""Legacy/Outback""",null,null,"""T""",null,null,null,null,null,null,null,0.0,null,"""Tue Jan 01 00:00:00 EST 2013""","""Tue Jan 01 00:00:00 EST 2013""",null,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
13.523182,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,"""Front-Wheel Drive""",66030,"""(FFS)""",-1,2200,0,"""Regular""","""Regular Gasoline""",-1,-1,26,0.0,0,…,"""Legacy""","""N""",false,0,90,0,0.0,0.0,0.0,0.0,"""Automatic 4-spd""",24.0,0.0,37.0,0.0,"""Compact Cars""",1993,-2250,"""Legacy/Outback""",null,"""CLKUP""",null,null,null,null,null,null,null,null,0.0,null,"""Tue Jan 01 00:00:0

In [27]:
print(df_polars.select(mean_ba=pl.col.barrels08.mean(),
                       std_ba=pl.col.barrels08.std(),
                      var_ba=pl.col.barrels08.var(),
                      q99_ba=pl.col.barrels08.quantile(.99)
 ))

shape: (1, 4)
┌───────────┬──────────┬───────────┬───────────┐
│ mean_ba   ┆ std_ba   ┆ var_ba    ┆ q99_ba    │
│ ---       ┆ ---      ┆ ---       ┆ ---       │
│ f64       ┆ f64      ┆ f64       ┆ f64       │
╞═══════════╪══════════╪═══════════╪═══════════╡
│ 15.049851 ┆ 4.623639 ┆ 21.378037 ┆ 27.046364 │
└───────────┴──────────┴───────────┴───────────┘


In [28]:
#One feature of Polars is the ability to have typed lists in a column
tests = pl.DataFrame({'name':['Tom', 'Sally', 'Jose'],
 'test1':[99, 98, 95],
 'test2':[92, None, 99],
 'test3':[91, 93, 95],
 'test4':[94, 92, 99]})
print(tests)

shape: (3, 5)
┌───────┬───────┬───────┬───────┬───────┐
│ name  ┆ test1 ┆ test2 ┆ test3 ┆ test4 │
│ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ str   ┆ i64   ┆ i64   ┆ i64   ┆ i64   │
╞═══════╪═══════╪═══════╪═══════╪═══════╡
│ Tom   ┆ 99    ┆ 92    ┆ 91    ┆ 94    │
│ Sally ┆ 98    ┆ null  ┆ 93    ┆ 92    │
│ Jose  ┆ 95    ┆ 99    ┆ 95    ┆ 99    │
└───────┴───────┴───────┴───────┴───────┘


In [32]:
import polars.selectors as cs
print(tests
 .select(scores=pl.concat_list(cs.matches(r'test\d+')))#combine the test scores into a list with the pl.concat_list function
 .with_columns(sorted_scores=pl.col('scores').list.sort())#sort the scores
 .with_columns(slice_scores=pl.col('sorted_scores').list.slice(2,4))#remove 2 lowest scores
 .with_columns(sum_scores=pl.col('slice_scores').list.sum())#sum up the list to get the sum of final score
 )

shape: (3, 4)
┌──────────────────┬──────────────────┬──────────────┬────────────┐
│ scores           ┆ sorted_scores    ┆ slice_scores ┆ sum_scores │
│ ---              ┆ ---              ┆ ---          ┆ ---        │
│ list[i64]        ┆ list[i64]        ┆ list[i64]    ┆ i64        │
╞══════════════════╪══════════════════╪══════════════╪════════════╡
│ [99, 92, … 94]   ┆ [91, 92, … 99]   ┆ [94, 99]     ┆ 193        │
│ [98, null, … 92] ┆ [null, 92, … 98] ┆ [93, 98]     ┆ 191        │
│ [95, 99, … 99]   ┆ [95, 95, … 99]   ┆ [99, 99]     ┆ 198        │
└──────────────────┴──────────────────┴──────────────┴────────────┘
